In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *


import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from tensorflow.keras.layers import InputLayer, BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random
from catboost import CatBoost



from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score


In [2]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np



In [3]:
path = "./data/"
trainX_raw, trainY_raw = load_data("", "train", path)
valX_raw, valY_raw = load_data("", "Val", path)

trainX_balanced, trainY_balanced = get_balanced_set(trainX_raw, trainY_raw)

valX, valY = valX_raw, valY_raw

features = trainX_raw.shape[-1]


In [4]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(features,))
normalizer.adapt(trainX_raw)

In [5]:
print(Counter(trainY_raw))


X_over, y_over = get_balanced_set(trainX_raw, trainY_raw)

print()
print("{} {}".format(Counter(y_over), Counter(trainY_raw)))

print("{} {} | {} {}".format(X_over.shape, y_over.shape, valX.shape, valY.shape))


Counter({0: 2548, 1: 1578})

Counter({1: 2548, 0: 2548}) Counter({0: 2548, 1: 1578})
(5096, 16) (5096,) | (460, 16) (460,)


In [6]:
%%time
from catboost import CatBoostClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score


X_train, Y_train = trainX_raw, trainY_raw

x_norm = normalizer(trainX_raw).numpy()

clf = CatBoostClassifier()

params = {'iterations': [500],
          'depth': [
                    5,
                    6,
                    10
          ],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10, 15, 20],
          'eval_metric': ['Precision'],
          #'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }


CPU times: user 3.56 ms, sys: 2.75 ms, total: 6.31 ms
Wall time: 3.7 ms


In [7]:
%%time


clf_grid = gridSearch(X_train, Y_train, clf, params)

#0.6743010531359825

{'depth': 10, 'eval_metric': 'Precision', 'iterations': 500, 'l2_leaf_reg': 3.162277660168379e-20, 'leaf_estimation_iterations': 10, 'logging_level': 'Silent', 'loss_function': 'Logloss', 'random_seed': 42}
0.6400768053573737
CPU times: user 1h 37min 51s, sys: 43min 17s, total: 2h 21min 9s
Wall time: 29min 32s


In [8]:
#model = CatBoostClassifier(**clf_grid.best_params_)

#model.fit(X_train, Y_train)
#model.fit(x_norm, Y_train)

model = clf_grid.best_estimator_

eval_data(model, X_train, Y_train)

#5473554735547356

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      2548
     class 1       1.00      1.00      1.00      1578

    accuracy                           1.00      4126
   macro avg       1.00      1.00      1.00      4126
weighted avg       1.00      1.00      1.00      4126



In [9]:
norm_val_x = normalizer(valX).numpy()

eval_data(model, valX, valY)


              precision    recall  f1-score   support

     class 0       0.79      0.85      0.82       294
     class 1       0.69      0.60      0.64       166

    accuracy                           0.76       460
   macro avg       0.74      0.72      0.73       460
weighted avg       0.75      0.76      0.75       460



In [10]:
from joblib import dump, load
dump(model, 'CatBoostClassifier') 

['CatBoostClassifier']

In [33]:
valX_raw, valY_raw = load_data("backtest", "train", path)

#eval_data(model, valX_raw, valY_raw)

for x in valX_raw:
    arrY = model.predict([x])
    if (arrY[0] == 1):
        print(np.array(x))

[ 1.66020987e+01  5.40415649e-01 -2.86489157e+00 -4.52950000e+01
 -1.29455250e+02  9.17832750e-01  9.98478757e-01 -6.39421636e+00
 -6.65658063e+00  1.97558075e+00  9.99472338e+01  2.49759576e-02
 -2.84761926e-02  2.49759576e-02 -2.40979650e-02 -9.18516198e-01]
[ 2.66790243e+01  5.85877751e-01 -1.18092753e+00 -1.10756000e+02
 -4.92577500e+01  8.34921162e-01  9.97731650e-01 -6.58898400e+00
 -5.85138505e+00  1.31339011e+01  9.99267531e+01  1.84303291e-01
  1.00581111e-01  1.84303291e-01  5.27591804e-01  8.78549867e-02]
[-9.22948182e+00  4.40712719e-01  1.75874372e+00  7.48725000e+01
  2.75620000e+01  2.99071647e-01  1.00121686e+00 -5.87401923e+00
 -6.37045611e+00 -8.59840638e+00  7.83430775e+00 -3.22299895e-02
 -4.05324040e-03 -3.22299895e-02 -1.02394392e-01  1.57572478e-01]
[-1.03132833e+01  4.63639649e-01  2.60709122e+00  3.77945000e+01
  3.00120000e+01  9.40821730e-02  1.00114267e+00 -9.01746684e+00
 -6.93802530e+00  4.08671210e+00  7.03937098e-02  2.53657078e-02
  5.66160410e-02  2.53

[-1.49091403e+01  4.44390978e-01  1.43652749e+00 -7.13560000e+01
  1.35438750e+02  2.21182904e-01  1.00177760e+00 -6.18796377e+00
 -6.01611351e+00 -2.12193856e+00  4.68296706e+01 -2.37586779e-02
  2.39687568e-02 -2.37586779e-02 -6.77595796e-01  3.42131365e-01]
[ 15.7170441    0.47130579  -5.76050207  91.269      -89.89025
   0.7343023    1.00000696  -6.63328159  -7.14410721  -7.83672485
  99.99999121  -0.16705531  -0.21839866  -0.16705531  -0.82585096
  -1.17498816]
[-5.46971330e+00  4.07930249e-01 -1.78685748e+00 -1.55795000e+01
  9.11135000e+01  1.72163059e-01  1.00064632e+00 -6.07402241e+00
 -6.93901985e+00 -1.58785875e+01  2.84153399e-02 -9.60505653e-02
 -7.94807824e-02 -9.60505653e-02 -5.22841355e-01 -6.48864495e-01]
[-12.85175813   0.56290746   5.85323775  45.4345     -72.455
   0.           1.0008036   -8.97807314  -6.33901581  28.57553616
   0.6002808    0.10877608   0.14943395   0.10877608   0.34407531
   0.84772931]
[-7.14931590e+00  4.31555913e-01  1.16158254e+00  3.15645000

[-5.74098789e+02  5.52580133e-01  8.83815113e+05 -1.28443000e+02
  3.09526750e+02  9.42826182e-01  9.97668198e-01 -6.42022198e+00
 -7.88655904e+00  4.75135881e+00  1.19209290e-05  4.70930097e+00
  6.91174846e+00  4.70930097e+00 -5.04065172e-01  5.72788510e+05]
[-6.02141646e+00  4.01360928e-01 -5.78468379e-01  1.06075000e+01
  8.37875000e+01  0.00000000e+00  1.00223655e+00 -5.79261751e+00
 -6.40572198e+00 -6.01906240e+00  3.63518017e+01 -4.25473828e-02
 -2.34605235e-02 -4.25473828e-02 -2.97313315e-01 -3.55895207e-01]
[-9.53454856e+00  3.34317284e-01 -8.37261062e-01  2.29660000e+01
  3.79147500e+01  1.23359073e-01  1.00112670e+00 -5.96275424e+00
 -6.70469158e+00 -2.43190884e+01  1.22068971e-02 -8.49656671e-02
 -5.58016694e-02 -8.49656671e-02 -2.50861870e-01 -2.94544289e-01]
[-1.85090290e+01  1.86372702e-01 -7.37149783e+00  2.44655000e+01
  1.48543250e+02  0.00000000e+00  1.00394018e+00 -5.82409596e+00
 -8.26644300e+00 -5.02913828e+01  1.36090265e-04 -3.12797504e-01
 -2.53340443e-01 -3.12

[ 2.63446519e+01  6.83739999e-01 -2.24431546e+00 -8.94740000e+01
 -5.53402500e+01  1.00000000e+00  9.91795109e-01 -7.06251598e+00
 -5.78955030e+00  1.16711612e+01  9.96093751e+01  1.32963126e-01
  5.16778347e-02  1.32963126e-01  1.40958158e+00  7.34150673e-01]
[-1.01281828e+01  3.78213878e-01 -1.15967039e+00  8.85640000e+01
  3.81022500e+01  3.83765227e-02  1.00263740e+00 -5.84565300e+00
 -8.33055965e+00 -1.50112065e+01  2.75750523e+00 -8.11978600e-02
 -5.07484153e-02 -8.11978600e-02 -4.42326434e-01 -3.30600601e-01]
[-1.58312510e+01  4.66356599e-01  2.74913456e+00 -1.96130000e+01
  8.15162500e+01  2.50027898e-01  1.00089673e+00 -5.91874447e+00
 -7.40034901e+00 -2.97363196e-01  1.90734863e-04 -3.73142767e-03
  4.45802431e-02 -3.73142767e-03 -5.63575483e-01  9.54329970e-01]
[-4.06149602e+01  4.71035202e-01  1.09315850e+01  1.32624000e+02
 -7.93627500e+01  1.19731500e-01  1.00353507e+00 -8.97351331e+00
 -6.48860666e+00  6.89472620e+00  6.20263939e-01  7.83410375e-02
  2.06923208e-01  7.83

[-4.88904426e+00  5.18896517e-01  2.89403719e+00 -2.97700000e+01
  1.93106750e+02  3.38253008e-01  9.99597982e-01 -6.65766118e+00
 -8.26709909e+00  4.24386628e+00  6.54951969e+01  6.47990292e-02
  8.05465305e-02  6.47990292e-02 -5.51087409e-01  6.35426891e-01]
[-2.94329653e+01  5.11445313e-01  1.09054585e+01  2.09693500e+02
 -5.08607500e+01  1.17428743e-01  1.00266419e+00 -8.99713942e+00
 -6.43219006e+00  1.27690584e+01  7.81249876e-01  1.24197460e-01
  2.15409480e-01  1.24197460e-01  3.33973154e-01  7.02649447e-01]
[  16.10655579    0.72390645    7.29292633 -147.328       -83.14625
    0.71655855    0.99227052   -8.99916615   -5.78029033   29.10985786
   74.23092241    0.29820366    0.24828231    0.29820366    1.47560395
    1.64481882]
[-1.77088926e+00  4.82204146e-01  1.07607898e+00  1.68400000e+01
  1.22537500e+01  1.19565480e-01  1.00076778e+00 -7.01774869e+00
 -6.01921986e+00  6.65088367e-01  9.99999936e+01  2.27431911e-03
  7.93485017e-03  2.27431911e-03 -2.26644594e-01 -2.53104

[ 3.34501908e+01  6.10812800e-01 -3.22789589e+00 -5.93655000e+01
 -1.27780250e+02  9.78085294e-01  9.95013498e-01 -6.42301654e+00
 -5.76909007e+00  1.13569013e+01  9.97067693e+01  1.60817906e-01
  5.57236102e-02  1.60817906e-01  7.76737463e-01 -1.55730848e-01]
[-12.18118274   0.20474991  -3.1081832   21.6455      68.92525
   0.           1.00308486  -5.83659034  -9.01464417 -39.30351297
   2.81307957  -0.15525152  -0.11827371  -0.15525152  -0.73813055
  -0.65439076]
[-2.19558473e+01  2.22057040e-01 -1.27564551e+01  5.10360000e+01
  5.32097500e+01  3.58219157e-01  1.00391681e+00 -5.81475039e+00
 -7.34080670e+00 -5.51469025e+01  2.36737794e-06 -5.52767744e-01
 -4.80988365e-01 -5.52767744e-01 -2.05835826e+00 -1.96750911e+00]
[ 3.70339065e+01  5.95194047e-01 -4.76583512e+00 -1.35079500e+02
 -1.26894250e+02  9.26068217e-01  9.94533056e-01 -6.27833348e+00
 -5.89534122e+00  8.35452848e+00  9.88280190e+01  1.20329236e-01
  3.85613014e-03  1.20329236e-01  7.92272779e-01 -4.58287549e-02]
[-1.716

[-7.74524077e+00  4.23717571e-01  6.29890211e-01 -2.86350000e+00
 -1.43580000e+02  0.00000000e+00  1.00187668e+00 -8.33276241e+00
 -6.62801432e+00 -6.81045758e-01  1.11081440e+00 -5.52979428e-03
  1.77070951e-02 -5.52979428e-03  2.10283401e-01 -3.41133524e-01]
[-1.07106696e+01  3.82753417e-01 -7.87048055e-01  1.09786000e+02
 -1.36266000e+02  0.00000000e+00  1.00301518e+00 -7.91935619e+00
 -6.37891115e+00 -1.02351430e+01  1.78101484e+00 -6.81901030e-02
 -3.58272667e-02 -6.81901030e-02  1.81548105e-02 -4.55911923e-01]
[-6.00449569e+01  3.17716111e-01  3.43050556e+00  2.27530000e+02
 -1.85103000e+02  0.00000000e+00  1.01142181e+00 -5.74631660e+00
 -6.32613510e+00 -1.26862191e+01  4.57418506e-01 -1.88136782e-01
  1.48047322e-03 -1.88136782e-01 -8.28636428e-01 -6.53013347e-01]
[  14.85714071    0.64427579    3.63310022 -164.077       112.18125
    0.95035338    0.99385633   -6.27733592   -5.76651029   18.16894924
   97.62814843    0.23899144    0.19240751    0.23899144    0.36359763
    1.5

[-5.46960135e+01  3.15299174e-01  7.34591861e+00  6.24580000e+01
  7.64705000e+01  0.00000000e+00  1.00784009e+00 -5.77241324e+00
 -6.42633970e+00 -6.41672686e+00  1.50272496e-02 -7.13034391e-02
  9.76139513e-02 -7.13034391e-02 -7.85441920e-01  7.36732836e-01]
[-8.35445784e+00  4.70623951e-01  4.31054860e-01  2.90065000e+01
 -1.43950000e+00  5.69924043e-02  1.00118317e+00 -8.30207888e+00
 -6.51031941e+00 -7.88958150e-03  1.61134111e+00 -5.43396524e-04
  2.53264954e-02 -5.43396524e-04 -1.77717126e-01 -4.05811653e-01]
[ 17.58496137   0.45206979 -10.0433143  -16.5085      48.78075
   0.54122011   1.00054773  -6.75789801  -7.8565103  -10.58337849
  98.82809187  -0.12505883  -0.18155353  -0.12505883  -0.25703441
  -0.86137924]
[-2.67763737e+02  2.92787722e-01 -3.31311744e+00 -4.74675000e+01
  1.08395000e+02  0.00000000e+00  1.00742192e+00 -5.74777576e+00
 -6.88721004e+00 -2.63490241e+01  7.27595761e-10  1.20357737e+00
  2.09188618e+00  1.20357737e+00 -1.77083017e+00 -1.78277094e+00]


              precision    recall  f1-score   support

     class 0       0.54      0.70      0.61       995
     class 1       0.45      0.30      0.36       830

    accuracy                           0.52      1825
   macro avg       0.50      0.50      0.49      1825
weighted avg       0.50      0.52      0.50      1825

